In [1]:
import numpy as np

import torch as th
print("Using pytorch {}".format(th.__version__))

import pandas as pd
from time import time

import matplotlib.pyplot as plt
%matplotlib inline

from utils import set_pretty_prints, load_dataset



Using pytorch 2.1.0


In [ ]:
set_pretty_prints()

In [ ]:
df = load_dataset('imobiliare.ro')

In [ ]:
df

In [ ]:
y_sqmp = df["Price/Surface"]
y_price = df['Price']

# TODO: select viable features
START = 1
END = 9
X = df.iloc[:,START:END]

In [ ]:
X

In [ ]:
y_price

## Exploration

In [ ]:
field = 'nr cam'
x_label = 'Nr rooms'
title = 'Distribution of nr of rooms per apartment'
X[field].hist(bins=20)
plt.xlabel(x_label)
plt.title(title)

In [ ]:
# TODO: Analyse a few more features
field = 'mp'
x_label = 'Square meters per property'
title = 'Distribution of sqm per property'
X[field].hist(bins=20)
plt.xlabel(x_label)
plt.title(title)

In [ ]:
# TODO: Analyse target distribution
target = y_price
title = 'Distribution of price'
x_label = 'Price'
plt.hist(target, bins=50)
plt.title(title)
plt.xlabel(x_label)

In [ ]:
# TODO: Re-display the target distribution
target = y_price
title = "Distribution of price in log scale"
x_label = "Price"
plot_param = 'log'
plot_param_value = True
kwargs = {plot_param : plot_param_value}
plt.hist(target, bins=50, **kwargs)
plt.title(title)
plt.xlabel(x_label)

In [ ]:
X.corr()

In [ ]:
new_corr_features = ['nr cam', 'mp', 'parter', 'et1-2', 'et3+','etaj max', 'typ_decom', 'bloc nou', 'Price']

In [ ]:
df_new = df[new_corr_features]

In [ ]:
df_new.corr()

## Modelling
- further process X data maybe
- construct normal eq and determine model coefs `(((XtX)^-1)Xt)y (y = x*w => w = y/x)`
- validate results (how, when)

`f(X) = y = X[0]*w[0] + X[1]*w[1] + .... X[N-1]*w[N-1] +X[N]*w[N] | X[N] == 1`

In [ ]:
X.mean()

In [ ]:
X.min()

In [ ]:
X.std()

In [ ]:
X.max()[:10]

In [ ]:
np_X = X.values

In [ ]:
np_X[:20]

In [ ]:
np_X.mean(axis=0)

In [ ]:
np_X_n = (np_X - np_X.mean(0)) / np_X.std(0)

In [ ]:
np_X_n[:20]

In [ ]:
np_y = y_price.values
np_y[:20]

In [ ]:
np_y_n = (np_y - np_y.min()) / (np_y.max() - np_y.min())
np_y_n[:20]

In [ ]:
np_y.min()


In [ ]:
np_y.max()

In [ ]:
y_norm_sub = np_y.min()
y_norm_div = np_y.max() - np_y.min()
y_test = np_y_n * y_norm_div + y_norm_sub
y_test[:20]

In [ ]:
# TODO: write normal eq for raw data
np_weights = np.linalg.pinv(np_X.T.dot(np_X)).dot(np_X.T).dot(np_y)

# TODO: write normal eq for normalized data
np_weights_n = np.linalg.pinv(np_X_n.T.dot(np_X_n)).dot(np_X_n.T).dot(np_y_n)


In [ ]:
np_weights

In [ ]:
np_weights_n

In [ ]:
#TODO: calc predictions for raw data model
np_y_preds = np_X.dot(np_weights)

#TODO: calc predictions for normalized data model
np_y_preds_n = np_X_n.dot(np_weights_n)

In [ ]:
np_y_preds[:20]

In [ ]:
plt.figure()
plt.hist(np_y_preds)
plt.title('Raw model predictions')
plt.figure()
plt.hist(np_y_preds_n)
plt.title('Normed data model predictions')

### One more model before testing results

Lets further improve model by adding bias

In [ ]:
ones = np.ones(shape=(np_X_n.shape[0], 1))
ones[:20]

In [ ]:
np_X_nb = np.concatenate((np_X_n, ones), axis=-1)
np_X_nb[:20]

In [ ]:
# TODO: calculate weights
np_weights_nb = np.linalg.inv(np_X_nb.T.dot(np_X_nb) + 0.02 * np.eye(np_X_nb.shape[1])).dot(np_X_nb.T).dot(np_y_n)
np_weights_nb

In [ ]:
# TODO: calculate predictions
np_y_preds_nb = np_X_nb.dot(np_weights_nb)
np_y_preds_nb[:20]

In [ ]:
plt.hist(np_y_preds_nb, bins=50)

In [ ]:
np_y_pred_price = np_y_preds
np_y_pred_n_price = np_y_preds_n * y_norm_div + y_norm_sub
np_y_pred_nb_price = np_y_preds_nb  * y_norm_div + y_norm_sub

### Now lets prepare some friendly calitative analysis outputs

Raw model

In [ ]:
df_result_raw = pd.DataFrame(
    {
        'GOLD' : y_price,
        'PRED' : np_y_pred_price.round(0),
    }
)
df_result_raw.head(10)
df_result_raw.tail(10)

Normed data model

In [ ]:
df_result_n = pd.DataFrame(
    {
        'GOLD' : y_price,
        'PRED' : np_y_pred_n_price.round(0),
    }
)
df_result_n.head(10)
df_result_n.tail(10)

Normed & bias added

In [ ]:

df_result_nb = pd.DataFrame(
    {
        'GOLD' : y_price,
        'PRED' : np_y_pred_nb_price.round(0),
    }
)
df_result_nb.head(10)
df_result_nb.tail(10)

Now lets see some quantitative analysis of the results

In [ ]:
# TODO: complete code below
abs_err = np.abs(y_price - np_y_pred_nb_price)
abs_err

In [ ]:
proc_err = abs_err / y_price
proc_err = proc_err * 100

In [ ]:
df_result = pd.DataFrame(
    {
        'GOLD' : y_price,
        'PRED' : np_y_pred_nb_price.round(0),
        'ERR%' : proc_err.round(2)
    }
)
df_result.head(20)

In [ ]:
df_result.tail(20)

In [ ]:
proc_err.mean()

In [ ]:
def train_neq(inputs, gold):
    # TODO:
    weights = np.linalg.pinv(inputs.T.dot(inputs)).dot(inputs.T).dot(gold)
    return weights

def evaluate(theta, inputs, gold, y_div, y_sub, name=""):
    _y_pred = inputs.dot(theta)
    _y_vals = _y_pred * y_div + y_sub
    
    _y_true = gold * y_div + y_sub
    
    res_err = np.abs(_y_true - _y_vals)
    prc_err = res_err / _y_true *100
    
    overall = prc_err.mean()
    df_result = pd.DataFrame(
        {
        'GOLD' : _y_true,
        'PRED' : _y_vals.round(0),
        'ERR%' : prc_err.round(2)
        }
    )
    print('Results for', name)
    print(df_result.head(20))
    print(df_result.tail(20))
    print("Overall error: {:.1f}%".format(overall))
    return overall

In [ ]:
# Note that Web Page format includes the area name.
df['Location'] = df['WebPage'].apply(lambda getloc : getloc.split('/')[5])
df

In [ ]:
len(df.Location.unique())

In [ ]:
df_locs= pd.get_dummies(df.Location, prefix='loc', columns=['Location'], dtype=float)
np_locs = df_locs.values

np_X_loc_n = np.concatenate((np_X_n, np_locs), axis=1)
np_X_loc_nb = np.concatenate((np_X_loc_n, ones), axis=1)
df_locs.head(10)

In [ ]:
np_X_loc_nb.shape

In [ ]:
print('No location data:\n{}'.format(np_X_nb[:5]))
print('\With location data:\n{}'.format(np_X_loc_nb[:5, :20]))

## Lets see some correlations !

In [ ]:
df_locs['Price'] = df.Price
df_locs.corr().iloc[-10:,-10:]

### Now we can train the model with location data.

In [ ]:
np_weights_loc_nb = train_neq(np_X_loc_nb, np_y_n)
np_weights_loc_nb[:10]

In [ ]:
evaluate(
    theta=np_weights_loc_nb,
    inputs=np_X_loc_nb,
    gold=np_y_n,
    y_div=y_norm_div,
    y_sub=y_norm_sub,
    name='TRAIN',
)

In [ ]:
print("The previous normalize-data with bias model error was: {:.2f}%".format(proc_err.mean()))

# Now for a more correct and real-life approach
We will not use the pre-processed data and perform a train-test split. There is no need for train-dev-test split as we do not have a training process to use the dev on.

In [ ]:
from sklearn.model_selection import train_test_split

np_x_loc_trn, np_x_loc_tst, np_y_trn, np_y_tst = train_test_split(np_X_loc_nb, np_y_n, test_size=0.2)
# but is this enough ... ?


## Lets do the custom split "dance"

In [ ]:
def train_test_split_idx(data_size, test_size):
  test_len = int(data_size * test_size)
  all_idx = np.arange(data_size)
  np.random.shuffle(all_idx)
  test_idx = all_idx[:test_len]
  train_idx = all_idx[test_len:]
  return train_idx, test_idx

In [ ]:
train_idx, test_idx = train_test_split_idx(np_X_loc_nb.shape[0], test_size=0.2)
print(train_idx.shape, test_idx.shape)

In [ ]:
lst_experiments =[
  {
    "train" : np_X[train_idx],
    "test" : np_X[test_idx],
    "name" : "Normal"
  },
  {
    "train" : np_X_n[train_idx],
    "test" : np_X_n[test_idx],
    "name" : "Normalized",
  },
  {
    "train" : np_X_nb[train_idx],
    "test" : np_X_nb[test_idx],
    "name" : "Normalized + Bias",
  },
  {
    "train" : np_X_loc_nb[train_idx],
    "test" : np_X_loc_nb[test_idx],
    "name" : "Normalized + Bias + Location",
  }
]

dct_results = {
  'Experiment' : [],
  'Train Score' : [],
  'Test Score' : []
}

for dct_experiment in lst_experiments:
  experiment_name = dct_experiment["name"]
  print("Running experiment {}".format(experiment_name), flush=True)
  np_x_train = dct_experiment["train"]
  np_x_test = dct_experiment["test"]
  np_y_train = np_y_n[train_idx]
  np_y_test = np_y_n[test_idx]
  np_theta = train_neq(np_x_train, np_y_train)
  train_score = evaluate(
      theta=np_theta,
      inputs=np_x_train,
      gold=np_y_train,
      y_div=y_norm_div,
      y_sub=y_norm_sub,
      name='TRAIN {}'.format(dct_experiment["name"]),
  )
  test_score = evaluate(
      theta=np_theta,
      inputs=np_x_test,
      gold=np_y_test,
      y_div=y_norm_div,
      y_sub=y_norm_sub,
      name='TEST {}'.format(dct_experiment["name"]),
  )
  dct_results["Train Score"].append(train_score)
  dct_results["Test Score"].append(test_score)
  dct_results["Experiment"].append(experiment_name)

df_result = pd.DataFrame(dct_results).sort_values(by="Test Score")
print(df_result)

# now lets serialize the results
df_result.to_csv('outputs/results.csv', index=False)

# Simple Neural model


In [ ]:
import torch as th

# A*B*C*D*E*F*G*H*I*J*K*L*M*N*O*P*Q*R*S*T*U*V*W*X*Y*Z === A*X 

class SimpleLinerRealEstateModel(th.nn.Module):
    # Parameters:
    # n_feats - number of input features
    # n_hid1 - number of output features in the first hidden layers
    def __init__(self, n_feats, n_hid1=32):
        super().__init__()
        self.n_feats = n_feats
        self.hidden1 = th.nn.Linear(n_feats, n_hid1)
        self.act1 = th.nn.ReLU()
        self.readout = th.nn.Linear(n_hid1, 1)
        return
    
    def forward(self, inputs):
        #############################
        # TODO: complete forward pass 
        #############################
        th_x = inputs
        th_x = self.hidden1(th_x)
        th_x = self.act1(th_x)
        th_out = self.readout(th_x)
        return th_out

In [ ]:
model = SimpleLinerRealEstateModel(198, 256)
model

In [ ]:
assert len(np.unique(np_X_loc_nb[:,-1])) ==  1
np.unique(np_X_loc_nb[:,-1])
np_x_train = np_X_loc_nb[train_idx, :-1]
np_x_test_full = np_X_loc_nb[test_idx, :-1]
print(np_x_train.shape, np_x_test_full.shape)

### Introducing "dev" dataset
Now we will have a training process so we need a dev dataset

In [ ]:
DEV_PRC = 0.5
DEV_SIZE = int(np_x_test_full.shape[0] * DEV_PRC)
np_x_dev = np_x_test_full[:DEV_SIZE,:]
np_x_test = np_x_test_full[DEV_SIZE:,:]
print(np_x_dev.shape, np_x_test.shape)

Now we tensorize but we eliminate the bias term 

In [ ]:
th_x_trn = th.tensor(np_x_train, dtype=th.float32)
th_x_dev = th.tensor(np_x_dev, dtype=th.float32)
th_x_test = th.tensor(np_x_test, dtype=th.float32)
print(th_x_trn.shape, th_x_dev.shape, th_x_test.shape)
th_x_trn[:10]

In [ ]:
th.cuda.get_device_name()

In [ ]:
th_x_train_slice_gpu = th_x_trn[:10].to(th.device('cuda'))
th_x_train_slice_gpu

In [ ]:
np_y_n.shape

In [ ]:
np_y_trn = np_y_n[train_idx]
np_y_tst = np_y_n[test_idx]
# split in dev-test
np_y_dev = np_y_tst[:DEV_SIZE].reshape(-1,1)
np_y_test = np_y_tst[DEV_SIZE:].reshape(-1,1)

np_y_trn = np_y_trn.reshape(-1,1)
np_y_trn[:10]

In [ ]:
#############################
# TODO: complete y tensors creation 
#############################
th_y_trn = th.tensor(np_y_trn, dtype=th.float32)
th_y_dev = th.tensor(np_y_dev, dtype=th.float32)
th_y_test = th.tensor(np_y_test, dtype=th.float32)
th_y_trn[:20]

In [ ]:
print(th_y_dev.shape)
print(th_x_dev.shape)
print(th_y_test.shape)
print(th_x_test.shape)
print(th_y_trn.shape)
print(th_x_trn.shape)

### Model training data feed
Now lets prepare the internal mechanics for data feeding in the model training process

In [ ]:
BATCH_SIZE = 16
th_ds = th.utils.data.TensorDataset(th_x_trn, th_y_trn)
th_dl = th.utils.data.DataLoader(th_ds, batch_size=BATCH_SIZE)
th_x_trn.shape

In [ ]:
for th_x_batch, th_y_batch in th_dl:
    break
print(th_x_batch.shape, th_y_batch.shape)
th_x_batch

Re-writing evaluation function

In [ ]:

def th_evaluate(m, th_inputs, gold, y_div, y_sub, name="", verbose=False):
  m.eval()
  with th.no_grad():
    #############################
    # TODO: complete yhat generation 
    #############################            
    _y_pred = m(th_inputs)
      
  _y_vals = _y_pred * y_div + y_sub
  
  _y_true = gold * y_div + y_sub
  
  res_err = th.abs(_y_true - _y_vals)
  prc_err = res_err / _y_true * 100
  
  overall = prc_err.mean()
  if verbose:
      df_result = pd.DataFrame(
          {
          'GOLD' : _y_true.cpu().numpy().ravel(),
          'PRED' : _y_vals.cpu().numpy().ravel().round(0),
          'ERR%' : prc_err.cpu().numpy().ravel().round(2)
          }
      )
      print('Results for', name)
      print(df_result.head(20))
      print(df_result.tail(20))    
  m.train()
  return overall

In [ ]:
loss_func = th.nn.MSELoss()
# optimizer: weights = weights - alpha * grads # alpha << 1
opt = th.optim.Adam(model.parameters(), lr=5e-5)
opt

In [ ]:
DEBUG = False
TOTAL_NR_EPOCHS = 100
# re-init model
model = SimpleLinerRealEstateModel(th_x_trn.shape[1], 256)
print(model)
opt = th.optim.Adam(model.parameters(), lr=1e-4)
best_dev_err = 10_000
wait_time = 0
max_nr_of_succesive_fails = 5
for epoch in range(TOTAL_NR_EPOCHS):
  if DEBUG and epoch >0:
      break
  for th_x_batch, th_y_batch in th_dl:
    # compute current inferred values with forward prop
    th_y_hat = model(th_x_batch)
    # compute loss (compare results with actual truth)
    th_loss = loss_func(input=th_y_hat, target=th_y_batch) #((th_y_hat - th_y_batch)**2).mean()
    # nullfy the gradients
    opt.zero_grad()
    # compute loss 1st derv wrt all model weights (grads)
    th_loss.backward()
    
    if DEBUG:
        th_param = next(model.parameters())
        print(th_param.grad)
        break
    
    # apply gradients to weights with a hopefully smart approach
    opt.step()
  #end current epoch
  if not DEBUG:
    # now we evaluate on TRAIN and DEV to see how good we are
    train_err = th_evaluate(
        m=model,
        th_inputs=th_x_trn,
        gold=th_y_trn,
        y_div=y_norm_div,
        y_sub=y_norm_sub,
        verbose=False,
        name='TRAIN @ Epoch {}'.format(epoch)
    )
    dev_err = th_evaluate(
        m=model,
        th_inputs=th_x_dev,
        gold=th_y_dev,
        y_div=y_norm_div,
        y_sub=y_norm_sub,
        verbose=False,
        name='DEV @ Epoch {}'.format(epoch)
    )
    if best_dev_err > dev_err:
        best_dev_err = dev_err
        wait_time = 0
        print("BEST MODEL @ Epoch {} - train err: {:.2f}%, dev err: {:.2f}% ".format(epoch, train_err, dev_err), flush=True)
    else:
        wait_time += 1
        if wait_time > max_nr_of_succesive_fails:
            print(f"Stopped training at epoch {epoch} !")
            break

if not DEBUG: 
  # finally we evaluate on TEST
  th_evaluate(
    m=model,
    th_inputs=th_x_test,
    gold=th_y_test,
    y_div=y_norm_div,
    y_sub=y_norm_sub,
    verbose=True,
    name='Final TEST'
  )


In [ ]:
dev_err

In [ ]:
t1 = th.arange(0, num_locations, 1).view(1,-1).repeat(th_x_trn.shape[0], 1)[:10, :17]
t2 = th_x_trn[:10, 100:117]
print(t1.shape, t2.shape)
print(t1)
print(t2)
(t1 * t2).sum(-1).reshape(-1,1)

In [ ]:
# Recompute data with locations IDs in order to move to location embeddings.
num_locations = df['Location'].nunique()

# Turn one hot location encoding into index encoding
def th_add_location_idx(th_x):
  # Use a mask of consecutive numbers in the one dimension
  mask = th.arange(0, num_locations, 1).view(1,-1).repeat(th_x.shape[0], 1)
  # Multiply with the one hot encoding to mask values not equal to our index
  mask = mask * th_x[:,8:]
  # Do a row sum to get the actual index.
  locs = th.sum(mask, 1).view(-1, 1)
  return th.cat((th_x[:,:8], locs), axis=1)

th_x_dev_embed = th_add_location_idx(th_x_dev)
th_x_trn_embed = th_add_location_idx(th_x_trn)
th_x_test_embed = th_add_location_idx(th_x_test)


In [ ]:
th_x_dev_embed.shape

In [ ]:
th_x_test_embed.shape

In [ ]:
th_x_trn_embed.shape

In [ ]:
th_x_dev_embed[:20, -3:]

In [ ]:
th_ds = th.utils.data.TensorDataset(th_x_trn_embed, th_y_trn)
th_dl = th.utils.data.DataLoader(th_ds, batch_size=BATCH_SIZE)

for th_x_batch, th_y_batch in th_dl:
  break
print(th_x_batch.shape, th_y_batch.shape)
th_x_batch[:, -4:]

In [ ]:
NUM_LOCATIONS = num_locations
N_INPUTS = 8 + 1 # 8 features + 1 location

In [ ]:
class MoreAdvancedRealEstate(th.nn.Module):
  def __init__(self, embed_size=5, hsize=32, num_locations=NUM_LOCATIONS):
    super().__init__()
    self.embed_size = embed_size
    self.embed = th.nn.Embedding(num_embeddings=num_locations, embedding_dim=embed_size)
    self.hidden1 = th.nn.Linear((N_INPUTS -1 + embed_size), hsize)
    self.act1 = th.nn.ReLU()
    self.readout = th.nn.Linear(hsize, 1)
    return
  
  def forward(self, inputs):
    th_x_feat_input = inputs[:, :-1]
    th_x_embd_input = inputs[:, -1].long()
    th_embeds = self.embed(th_x_embd_input)
    th_x = th.cat((th_x_feat_input, th_embeds), axis=-1)
    th_x = self.hidden1(th_x)
    th_x = self.act1(th_x)
    th_out = self.readout(th_x)
    return th_out
  
  def get_embeds(self, inputs):
    th_x_embd_input = inputs[:, -1].long()
    th_embeds = self.embed(th_x_embd_input)
    return th_embeds

In [ ]:
model2 = MoreAdvancedRealEstate(embed_size=5, hsize=32)
model2

In [ ]:
th_x_batch.shape

In [ ]:
model2.get_embeds(th_x_batch)

In [ ]:
print("Initial model:\n{}\nModel Error: {}".format(model, dev_err))

In [ ]:
embed_size = NUM_LOCATIONS ** (1/3)
embed_size

In [ ]:
try:
  del model
except:
  pass
DEBUG = False
TOTAL_NR_EPOCHS = 100
# re-init model
model2 = MoreAdvancedRealEstate(embed_size=5, hsize=256)
print(model2)
opt = th.optim.Adam(model2.parameters(), lr=1e-4)
best_dev_err = 10_000
wait_time = 0
max_nr_of_succesive_fails = 5
for epoch in range(TOTAL_NR_EPOCHS):
  if DEBUG and epoch >0:
      break
  for th_x_batch, th_y_batch in th_dl:
    # compute current inferred values with forward prop
    th_y_hat = model2(th_x_batch)
    # compute loss (compare results with actual truth)
    th_loss = loss_func(input=th_y_hat, target=th_y_batch) #((th_y_hat - th_y_batch)**2).mean()
    # nullfy the gradients
    opt.zero_grad()
    # compute loss 1st derv wrt all model weights (grads)
    th_loss.backward()
    
    if DEBUG:
        th_param = next(model2.parameters())
        print(th_param.grad)
        break
    
    # apply gradients to weights with a hopefully smart approach
    opt.step()
  #end current epoch
  if not DEBUG:
    # now we evaluate on TRAIN and DEV to see how good we are
    train_err = th_evaluate(
        m=model2,
        th_inputs=th_x_trn_embed,
        gold=th_y_trn,
        y_div=y_norm_div,
        y_sub=y_norm_sub,
        verbose=False,
        name='TRAIN @ Epoch {}'.format(epoch)
    )
    dev_err = th_evaluate(
        m=model2,
        th_inputs=th_x_dev_embed,
        gold=th_y_dev,
        y_div=y_norm_div,
        y_sub=y_norm_sub,
        verbose=False,
        name='DEV @ Epoch {}'.format(epoch)
    )
    if best_dev_err > dev_err:
        best_dev_err = dev_err
        wait_time = 0
        print("BEST MODEL2 @ Epoch {} - train err: {:.2f}%, dev err: {:.2f}% ".format(epoch, train_err, dev_err), flush=True)
    else:
        wait_time += 1
        if wait_time > max_nr_of_succesive_fails:
            print(f"Stopped training at epoch {epoch} !")
            break

if not DEBUG: 
  # finally we evaluate on TEST
  th_evaluate(
    m=model2,
    th_inputs=th_x_test_embed,
    gold=th_y_test,
    y_div=y_norm_div,
    y_sub=y_norm_sub,
    verbose=True,
    name='Final TEST'
  )


In [ ]:
  th_evaluate(
    m=model2,
    th_inputs=th_x_test_embed,
    gold=th_y_test,
    y_div=y_norm_div,
    y_sub=y_norm_sub,
    verbose=True,
    name='Final TEST'
  )